In [2]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import tqdm

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy import exp
from scipy.optimize import curve_fit

import pandas as pd
import os
import shutil
import cv2

import glob2

from numba import jit
import tables
from scipy.io import loadmat
import scipy
import h5py
import hdf5storage
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.cm as cm

from utility_simulated_drift import (make_shifted_templates,
                                      make_default_template,
                                      draw_template,
                                      gaus,
                                      make_drift_template,
                                      shift_templates_all,
                                      WaveForms,
                                      generate_poisson_uniform_firingrate2,
                                      select_ground_truth_units,
                                      visualize_drift,
                                      generate_synthetic_data,
                                      visualize_traces,
                                      make_full_data)

colors = [
'black','grey','brown','slategrey',    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
clrs= colors


Autosaving every 180 seconds


In [3]:
# select working directory location
root_dir = '/media/cat/1TB/data/synthetic/run11/'
try:
    os.mkdir(root_dir)
except:
    pass

# set radius of channels to consider from maximum channel outwards (in micrometers)
radius = 200

# select ground truth templates to be used
#fname_templates = '/media/cat/12TB/dbox/Dropbox/code/neuropixels/allen_inst_2ndrec/templates_reloaded.npy'
#geom = np.loadtxt('/home/cat/p1_g0_t0.imec0.ap_geom_384chans.txt')
fname_templates = '/media/cat/1TB/data/synthetic/allen_inst_2ndrec/templates_reloaded.npy'
geom = np.loadtxt('/media/cat/1TB/data/synthetic/p1_g0_t0.imec0.ap_geom.txt')



# select largest X templates to be used for injection
n_templates = 100
(units, templates_ground_truth) = select_ground_truth_units(root_dir, 
                                                            fname_templates,
                                                            n_templates,
                                                            geom)

print ("templates ground truth: ", templates_ground_truth.shape)

ground truth directory already made
Total templates avialable;  (575, 101, 384)
Total units selected:  100 , ids:  [420 339 421 348 473 418  62 362 565 239] ...
templates ground truth:  (100, 101, 384)


In [3]:
# visualize the desired shifts on a particular unit
shifts = np.arange(0, 1., 0.05)

# visualize drifted template for any particular unit 
unit_id = 10
visualize_drift(shifts, geom, templates_ground_truth, unit_id,
               radius)


In [4]:
# generate synthetic drift data
sample_rate = 30000
rec_len_sec = 10
rec_length_sample_times = rec_len_sec * sample_rate

# spikes, scales = generate_poisson_uniform_firingrate2(rec_length)
# setup shifts
shifts = np.arange(0, .1, 0.05)
print ("shifts: ", shifts)

data_synthetic = generate_synthetic_data(root_dir,
                                         rec_len_sec, 
                                         sample_rate, 
                                         shifts, 
                                         units, 
                                         templates_ground_truth,
                                         radius,
                                         geom)

np.savetxt(root_dir+'/ground_truth/shifts.txt', shifts)
print (data_synthetic.shape)


shifts:  [0.   0.05]
(300000, 384)
[     0. 300000.]
window:  [0.0, 300000.0] , shift:  0.0  (inter channel units)
 inserting unit:  0 0
 inserting unit:  50 50
************


(300000, 384)


In [5]:
# visualize the final traces
visualize_traces(data_synthetic, 0, 5000)


In [5]:
# Make a correlated data chunk of 10 seconds 
# longer takes too long of time; shorter will add more boundary artifcats to recording
time = 10*sample_rate
print ("length of correlated noise snipit: ", time)

fname = root_dir+'correlated_noise.npy'
# use ground_truth_templates to make noise
if os.path.exists(fname)==False:
    temps = templates_ground_truth.copy()
    WF = WaveForms(temps.transpose(0,2,1))
    correlated_noise = WF.generate_correlated_noise_add(time)

    print ("DONE MAKING CORRELATED NOISE for 10 sec chunk")
    
    np.save(fname, correlated_noise)
else:
    correlated_noise = np.load(fname)

length of correlated noise snipit:  300000


In [6]:
# blend correlated noise with injected spike dataset
data_sum = make_full_data(data_synthetic, rec_length_sample_times, 
                          sample_rate, root_dir,
                          correlated_noise)


white noise:  (300000, 384)
time chunk:  0  to  300000


In [6]:
# visualize first 10000 time points;
visualize_traces(data_sum, 0, 10000)


In [7]:
# visualize spike triggered max chan data
spike_train_gt = np.load('/media/cat/1TB/data/synthetic/run11/ground_truth/spike_train_ground_truth.npy')
fname= '/media/cat/1TB/data/synthetic/run11/data_int16.bin'

from utility_match_units import binary_reader_waveforms

In [14]:
n_channels = 384
n_times = 101

# 
units = np.arange(25)
units = [13]
for unit in units:
    #ax=plt.subplot(5,5,unit+1)
    idx = np.where(spike_train_gt[:,1]==unit)[0]
    spikes = spike_train_gt[idx,0]
    wfs = binary_reader_waveforms(fname, n_channels, n_times, spikes, data_type='int16')

    print (wfs.shape)
    max_chan = wfs.mean(0).ptp(0).argmax(0)

    #fig=plt.figure()
    plt.plot(wfs[:,:,max_chan].T,c='black')
    plt.show()

    if True:
        fig=plt.figure()
        for k in range(wfs.shape[0]):
            ax=plt.subplot(10,10,k+1)
            plt.plot(wfs[k,:,max_chan].T,c='black')
            plt.title(str(k)+" "+str(spikes[k]))
        plt.show()

(85, 101, 384)


In [15]:
times = np.load('/media/cat/1TB/temp/'+str(unit)+'_times.npy')
scale = np.load('/media/cat/1TB/temp/'+str(unit)+'_scale.npy')

print (times.shape)
print (scale.shape)
print (scale)


(85,)
(85,)
[1.         0.69767633 0.64403642 1.         1.         0.99501248
 0.74453159 0.71177032 0.68045064 1.         1.         0.89136614
 0.79057085 1.         1.         1.         0.68728928 1.
 0.81873075 1.         1.         0.93706746 0.86070798 0.83945702
 1.         0.97044553 0.72614904 0.86935824 0.85641518 0.98019867
 1.         0.66034028 1.         0.62188506 1.         0.99004983
 0.69073433 0.77105159 0.83110428 0.70117344 1.         0.99004983
 1.         0.88692044 0.81058425 1.         0.61878339 0.92774349
 1.         0.63762815 1.         0.6156972  1.         1.
 0.68728928 0.78662786 1.         0.60957091 0.60957091 0.77880078
 0.85214379 1.         1.         1.         1.         0.72614904
 0.70117344 1.         1.         1.         0.66697681 0.99501248
 1.         1.         0.62188506 0.81873075 0.62500227 0.66034028
 0.65376979 1.         0.82695913 0.64082428 0.64082428 1.
 1.        ]
